In [ ]:
import os
import random
from typing import List
import more_itertools

from embedding_comparator import EmbeddingComparator

from embedding_generator import EmbeddingGenerator
from models import CodeCommentPair
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import orjson
from transformers import AutoTokenizer, TFAutoModel, logging, AutoConfig
logging.set_verbosity_error()

In [ ]:
negative_samples_count = 1
batch_size = 64
seq_len = 512
hidden_size = 768
dataset_len = {
    "test": {
        "java": 26880,
        "python": 22144
    },
    "train": {
        "java": 256,
        "python": 256
    },
    "valid": {
        "java": 15296,
        "python": 23104
    }
}